In this notebook, we will use perform hyper parameter tuning of a neural network with grid search.

In [ ]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetRegressor

In [ ]:
# Load the California Housing dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

In [ ]:
X.shape, y.shape

((20640, 8), (20640,))

In [ ]:
print(housing.feature_names)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [ ]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# create model with skorch
model = NeuralNetRegressor(
    RegressionModel,
    criterion=nn.MSELoss,
    optimizer=optim.Adam,
    verbose=False
)

In [ ]:
model.set_params(train_split=False, verbose=0)
param_grid = {
    'max_epochs': [1, 2],
    'lr': [0.01, 0.1], #[0.001, 0.005, 0.01, 0.05, 0.1],
    'batch_size': [8, 32] #[8, 32, 64, 128, 256]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid.fit(X_train_tensor, y_train_tensor)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
print(grid.best_score_, grid.best_params_)

0.13502475212623363 {'batch_size': 8, 'lr': 0.01, 'max_epochs': 2}


In [ ]:
grid.cv_results_

{'mean_fit_time': array([5.11460638, 6.8939383 , 3.41022515, 7.2747968 , 0.94423834,
        1.82287892, 1.25997376, 1.87440093]),
 'std_fit_time': array([1.37516314, 0.5178979 , 0.24316378, 0.47778399, 0.00993003,
        0.03720657, 0.24561108, 0.50147937]),
 'mean_score_time': array([0.56662035, 0.46225031, 0.43085766, 0.4373711 , 0.1906174 ,
        0.21317228, 0.28664454, 0.16376392]),
 'std_score_time': array([0.09755873, 0.03743579, 0.01534669, 0.00997535, 0.00232365,
        0.04288649, 0.03798626, 0.04552409]),
 'param_batch_size': masked_array(data=[8, 8, 8, 8, 32, 32, 32, 32],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_lr': masked_array(data=[0.01, 0.01, 0.1, 0.1, 0.01, 0.01, 0.1, 0.1],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_epochs': masked_array(data=[1, 2, 1, 2, 1, 2, 1, 2],
     

## Verdict

We can change the `param_grid` to include other hyper parameters like activation functions, dropout rate and more.

## Resources

1. Scorch documentation: https://skorch.readthedocs.io/en/stable/user/quickstart.html#grid-search
2. Sklearn GridSearchCV documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
3. Link to housing dataset documentation: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html
